# Libraries

In [3]:
import pandas as pd
import numpy as np
import csv
import multiprocessing
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from time import time  # To time our operations

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# Training Pipeline

In [7]:
cores = multiprocessing.cpu_count()   # Number of CPU cores used for training
t = time()  # Time of the process

# Looping through different window sizes
for wnd in range(10, 100, 10):
    print('Training model k:', wnd)

    file_name = './geographic/SHORTEST PATH CORPUS/austin-shortest_path_corpus-zoning-k' + str(wnd) + '-FT_gpu.csv'

    print("Loading file:", file_name)
    tuples = pd.read_parquet(file_name)
    # Removing damaged rows
    tuples = tuples.dropna()
    tuples = tuples[['center_poi', 'context_poi']]
    sentences = tuples.values.tolist()

    # Creating skip-gram model
    p2v_modeltp = Word2Vec(min_count=1,
                           window=1,
                           sg=1,  # CBOW = 0, SKIPGRAM = 1
                           size=70,
                           sample=6e-5,
                           alpha=0.03,
                           min_alpha=0.0007,
                           negative=20,
                           workers=cores-1)

    # Building vocabulary
    p2v_modeltp.build_vocab(sentences, progress_per=10000)

    # Training the model
    p2v_modeltp.train(sentences, total_examples=p2v_modeltp.corpus_count, epochs=20, report_delay=1)

    model_name = './geographic/SHORTEST PATH CORPUS/austin-shortest_path_corpus-zoning-k' + str(wnd) + '-FT.model'

    print('Saving file:', model_name)
    p2v_modeltp.save(model_name)

Training model bin: 2
loading file: ./geographic/SHORTEST PATH CORPUS/austin-shortest_path_corpus-2-CF-c_gpu.parquet


INFO - 11:18:17: collecting all words and their counts
INFO - 11:18:17: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:18:17: PROGRESS: at sentence #10000, processed 20000 words, keeping 492 word types
INFO - 11:18:18: PROGRESS: at sentence #20000, processed 40000 words, keeping 641 word types
INFO - 11:18:18: PROGRESS: at sentence #30000, processed 60000 words, keeping 711 word types
INFO - 11:18:18: PROGRESS: at sentence #40000, processed 80000 words, keeping 763 word types
INFO - 11:18:18: PROGRESS: at sentence #50000, processed 100000 words, keeping 803 word types
INFO - 11:18:18: PROGRESS: at sentence #60000, processed 120000 words, keeping 834 word types
INFO - 11:18:18: PROGRESS: at sentence #70000, processed 140000 words, keeping 858 word types
INFO - 11:18:18: PROGRESS: at sentence #80000, processed 160000 words, keeping 879 word types
INFO - 11:18:18: PROGRESS: at sentence #90000, processed 180000 words, keeping 896 word types
INFO - 11:18:18: PRO

INFO - 11:18:18: PROGRESS: at sentence #850000, processed 1700000 words, keeping 1116 word types
INFO - 11:18:18: PROGRESS: at sentence #860000, processed 1720000 words, keeping 1117 word types
INFO - 11:18:18: PROGRESS: at sentence #870000, processed 1740000 words, keeping 1118 word types
INFO - 11:18:18: PROGRESS: at sentence #880000, processed 1760000 words, keeping 1118 word types
INFO - 11:18:18: PROGRESS: at sentence #890000, processed 1780000 words, keeping 1120 word types
INFO - 11:18:18: PROGRESS: at sentence #900000, processed 1800000 words, keeping 1120 word types
INFO - 11:18:18: PROGRESS: at sentence #910000, processed 1820000 words, keeping 1122 word types
INFO - 11:18:18: PROGRESS: at sentence #920000, processed 1840000 words, keeping 1122 word types
INFO - 11:18:18: PROGRESS: at sentence #930000, processed 1860000 words, keeping 1122 word types
INFO - 11:18:18: PROGRESS: at sentence #940000, processed 1880000 words, keeping 1122 word types
INFO - 11:18:18: PROGRESS: at 

INFO - 11:18:18: PROGRESS: at sentence #1690000, processed 3380000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1700000, processed 3400000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1710000, processed 3420000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1720000, processed 3440000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1730000, processed 3460000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1740000, processed 3480000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1750000, processed 3500000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1760000, processed 3520000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1770000, processed 3540000 words, keeping 1137 word types
INFO - 11:18:18: PROGRESS: at sentence #1780000, processed 3560000 words, keeping 1137 word types
INFO - 11:18:18: PRO

INFO - 11:18:18: PROGRESS: at sentence #2530000, processed 5060000 words, keeping 1141 word types
INFO - 11:18:18: PROGRESS: at sentence #2540000, processed 5080000 words, keeping 1141 word types
INFO - 11:18:18: PROGRESS: at sentence #2550000, processed 5100000 words, keeping 1141 word types
INFO - 11:18:18: PROGRESS: at sentence #2560000, processed 5120000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2570000, processed 5140000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2580000, processed 5160000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2590000, processed 5180000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2600000, processed 5200000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2610000, processed 5220000 words, keeping 1142 word types
INFO - 11:18:18: PROGRESS: at sentence #2620000, processed 5240000 words, keeping 1142 word types
INFO - 11:18:18: PRO

...

INFO - 11:49:12: EPOCH 12 - PROGRESS: at 18.63% examples, 704291 words/s, in_qsize 38, out_qsize 3
INFO - 11:49:13: EPOCH 12 - PROGRESS: at 23.67% examples, 716574 words/s, in_qsize 37, out_qsize 0
INFO - 11:49:14: EPOCH 12 - PROGRESS: at 28.48% examples, 718712 words/s, in_qsize 38, out_qsize 0
INFO - 11:49:15: EPOCH 12 - PROGRESS: at 32.90% examples, 711968 words/s, in_qsize 37, out_qsize 0
INFO - 11:49:16: EPOCH 12 - PROGRESS: at 37.76% examples, 714037 words/s, in_qsize 36, out_qsize 6
INFO - 11:49:17: EPOCH 12 - PROGRESS: at 42.45% examples, 712351 words/s, in_qsize 38, out_qsize 0
INFO - 11:49:18: EPOCH 12 - PROGRESS: at 46.17% examples, 698361 words/s, in_qsize 37, out_qsize 0
INFO - 11:49:19: EPOCH 12 - PROGRESS: at 50.61% examples, 695403 words/s, in_qsize 37, out_qsize 0
INFO - 11:49:20: EPOCH 12 - PROGRESS: at 55.58% examples, 699997 words/s, in_qsize 38, out_qsize 0
INFO - 11:49:21: EPOCH 12 - PROGRESS: at 60.07% examples, 698899 words/s, in_qsize 37, out_qsize 0
INFO - 11:

INFO - 11:50:05: EPOCH 14 - PROGRESS: at 64.94% examples, 701923 words/s, in_qsize 38, out_qsize 1
INFO - 11:50:06: EPOCH 14 - PROGRESS: at 69.84% examples, 704849 words/s, in_qsize 38, out_qsize 1
INFO - 11:50:07: EPOCH 14 - PROGRESS: at 74.55% examples, 704488 words/s, in_qsize 35, out_qsize 4
INFO - 11:50:08: EPOCH 14 - PROGRESS: at 79.64% examples, 708244 words/s, in_qsize 37, out_qsize 0
INFO - 11:50:09: EPOCH 14 - PROGRESS: at 84.89% examples, 712522 words/s, in_qsize 38, out_qsize 0
INFO - 11:50:10: EPOCH 14 - PROGRESS: at 89.57% examples, 712470 words/s, in_qsize 37, out_qsize 0
INFO - 11:50:11: EPOCH 14 - PROGRESS: at 94.37% examples, 713781 words/s, in_qsize 37, out_qsize 0
INFO - 11:50:12: EPOCH 14 - PROGRESS: at 99.39% examples, 715517 words/s, in_qsize 35, out_qsize 2
INFO - 11:50:12: worker thread finished; awaiting finish of 18 more threads
INFO - 11:50:12: worker thread finished; awaiting finish of 17 more threads
INFO - 11:50:12: worker thread finished; awaiting finish

INFO - 11:50:54: worker thread finished; awaiting finish of 15 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 14 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 13 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 12 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 11 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 10 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 9 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 8 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 7 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 6 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 5 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 4 more threads
INFO - 11:50:54: worker thread finished; awaiting finish of 3 more threads
INFO - 11:50:54: wo

INFO - 11:51:36: worker thread finished; awaiting finish of 0 more threads
INFO - 11:51:36: EPOCH - 18 : training on 70972642 raw words (15169403 effective words) took 21.1s, 720503 effective words/s
INFO - 11:51:37: EPOCH 19 - PROGRESS: at 4.04% examples, 603072 words/s, in_qsize 37, out_qsize 0
INFO - 11:51:38: EPOCH 19 - PROGRESS: at 8.51% examples, 637316 words/s, in_qsize 37, out_qsize 2
INFO - 11:51:39: EPOCH 19 - PROGRESS: at 13.37% examples, 676002 words/s, in_qsize 37, out_qsize 0
INFO - 11:51:40: EPOCH 19 - PROGRESS: at 17.84% examples, 675403 words/s, in_qsize 36, out_qsize 1
INFO - 11:51:41: EPOCH 19 - PROGRESS: at 22.25% examples, 674451 words/s, in_qsize 37, out_qsize 1
INFO - 11:51:42: EPOCH 19 - PROGRESS: at 27.07% examples, 682635 words/s, in_qsize 38, out_qsize 0
INFO - 11:51:43: EPOCH 19 - PROGRESS: at 32.32% examples, 698931 words/s, in_qsize 38, out_qsize 3
INFO - 11:51:44: EPOCH 19 - PROGRESS: at 37.24% examples, 704110 words/s, in_qsize 38, out_qsize 0
INFO - 11:

Process finish.


In [8]:
p2v_modeltp.wv.most_similar(positive=['Zoos'])

INFO - 21:36:36: precomputing L2-norms of word weight vectors


[('Gutter Services', 0.9558095932006836),
 ('Plumbing', 0.9509575366973877),
 ('Septic Services', 0.949760913848877),
 ('Churches', 0.9384828805923462),
 ('Flea Markets', 0.932166576385498),
 ('Marinas', 0.9298552870750427),
 ('Water Heater Installation/Repair', 0.9289876818656921),
 ('Bus Rental', 0.925334095954895),
 ('Amusement Parks', 0.9248497486114502),
 ('Hydro-jetting', 0.9221491813659668)]